In [1]:
from tqdm import tqdm

from modules.stable_diffusion_webui import xy_script_args, txt2img_base
from modules.metadata import chunks
from modules.sd_api import find_model


API = 'http://127.0.0.1:7860/sdapi/v1'


In [3]:
artist_file = 'E:/ai/PDArtistGrid.txt'
AllArtists = open(artist_file, mode="r", encoding='utf-8').read().split("\n")

In [4]:
# Change models/options
model_name = find_model("awpmixXL")
print(model_name)
requests.post(url=f'{API}/options', json={"sd_model_checkpoint": model_name})



SDXL\awpmixXL_v10


In [ ]:

import json
prompt_base = json.load(open('prompt_base.json'))
prompt = prompt_base['prompt']
negative_prompt = prompt_base['negative_prompt']
seed = prompt_base['seed']
sampler = prompt_base['sampler']
width = prompt_base['width']
height = prompt_base['height']

payload = {**txt2img_base}
payload['negative_prompt'] = negative_prompt
payload['seed'] = seed
payload['save_images'] = True
payload['send_images'] = True
payload['steps'] = 30
payload['sampler'] = sampler
payload['batch_size'] = 4
payload['width'], payload['height'] = width, height





In [ ]:
######################################
# Set up xyzplot
######################################
xyz_payload = {**payload}
xyz_payload["script_name"] = "x/y/z grid"
xyz_payload["override_settings"] = {'samples_save': False, 'n_rows': 1}
xyz_payload['send_images'] = False  # We don't need em in Python

AllTargetLists = [nfArtists]

if prompt.find(ARTIST) == -1:
    raise Exception("Artist not found!")
for TargetArtistList in AllTargetLists:
    # Send batches
    for batch in tqdm(list(chunks(TargetArtistList, 25))):
        print(batch)
        xyz_payload['prompt'] = prompt.replace(ARTIST, batch[0])
        xyz_payload["script_args"] = xy_script_args(YAxisType="Prompt S/R", YAxisValues=",".join(batch))
        response = requests.post(url=f'{API}/txt2img', json=xyz_payload)
